In [1]:
!pip install gpytorch

Cloning into 'TuRBO'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 25 (delta 4), reused 2 (delta 2), pack-reused 12
Receiving objects: 100% (25/25), 70.05 KiB | 779.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 8.3 MB/s eta 0:00:00


In [2]:
from google.colab import files

In [3]:
!pip install cocopp
!git clone https://github.com/numbbo/coco.git
!cd coco; python do.py run-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 16.4 MB/s eta 0:00:00
Cloning into 'coco'...
remote: Enumerating objects: 57651, done.
remote: Counting objects: 100% (3506/3506), done.
remote: Compressing objects: 100% (1251/1251), done.
remote: Total 57651 (delta 2386), reused 3108 (delta 2157), pack-reused 54145
Receiving objects: 100% (57651/57651), 277.60 MiB | 22.70 MiB/s, done.
Resolving deltas: 100% (37262/37262), done.
AML	['code-experiments/src/coco_random.c', 'code-experiments/src/coco_suite.c', 'code-experiments/src/coco_observer.c', 'code-experiments/src/coco_archive.c', 'code-experiments/src/coco_runtime_c.c'] -> code-experiments/build/python/cython/coco.c
EXPAND	code-experiments/build/python/cython/coco.c.in to code-experiments/build/python/cython/coco.c
EXPAND	code-experiments/src/coco.h to code-experiments/build/python/cython/coco.h
COPY	code-experiments/src/bbob2009_testcases.txt -> code-experiments/build/python/bbob2009_testcases.txt
COPY	code-experiments/src

In [7]:
!cd coco/code-experiments/build/python; git clone https://github.com/uber-research/TuRBO

Cloning into 'TuRBO'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 25 (delta 4), reused 2 (delta 2), pack-reused 12
Receiving objects: 100% (25/25), 70.05 KiB | 797.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.


In [11]:
!cd coco/code-experiments/build/python; python example_experiment_for_beginners.py

COCO INFO: Results will be output to folder exdata/scipy-optimize-fmin-003
2D 14h28:35s
.......... .......... .......... .......... ..........
.......... |...|..... .......... .......... ..........
.......... .......... .......... .......... ..........
.......... .......... .......... .......... ..........
.......... .......... .......... .......... ..........
.......... .......... .......... .......... ..........
.......... .......... .......... .......... ..........
..........
3D 15h59:15s
.......... .......... .......... .......... ..........
.......... |...|..... .........Traceback (most recent call last):
  File "/content/coco/code-experiments/build/python/example_experiment_for_beginners.py", line 59, in <module>
    fmin(problem, x0, verbose=False)  # here we assume that `fmin` evaluates the final/returned solution
  File "/content/coco/code-experiments/build/python/example_experiment_for_beginners.py", line 39, in fmin
    return fmin_.optimize()
  File "/content/coco/code-expe